In [ ]:
import ROOT
ROOT.RooMsgService.instance().setGlobalKillBelow(5)
import time
import matplotlib

In [ ]:
infile = ROOT.TFile.Open("output/workspace1channels1000events1bins0nps.root")
w = infile.Get("BinnedWorkspace")
mc = w.obj("ModelConfig")
pdfsum = mc.GetPdf() 
data = w.data("obsData")
x = w.var("obs_x_Region0")
mc.LoadSnapshot()

In [ ]:
nll = pdfsum.createNLL(data, ROOT.RooFit.NumCPU(2,0))

In [ ]:
m  = ROOT.RooMinuit(nll)
m.migrad()
m.hesse()
m.minos(mc.GetParametersOfInterest())

In [ ]:
def NLL(ws, N_CPUs):
    data = ws.data("obsData")
    pdf = ws.obj("ModelConfig").GetPdf()
    nll = pdf.createNLL(data, ROOT.RooFit.NumCPU(2,0)) 
    m  = ROOT.RooMinuit(nll)
    m.migrad()
    m.hesse()
    m.minos(ws.obj("ModelConfig").GetParametersOfInterest())
    return nll

In [ ]:
eventtimes = []
for events in [10,100,1000,10000,100000,1000000,10000000]:
    av_time = []
    infile = ROOT.TFile.Open("output/workspace{}channels{}events{}bins{}nps.root".format(1, events, 1, 0))
    w = infile.Get("BinnedWorkspace")
    start = time.time()
    NLL(w,1)
    end = time.time()
    av_time.append(float(end) - float(start))
    eventtimes.append([sum(av_time)/float(len(av_time)),events])
    infile.Close()

In [ ]:
bintimes = []
for bins in [1,10,20,30,40,50,60,70,80,90,100]:
    av_time = []
    infile = ROOT.TFile.Open("output/workspace{}channels{}events{}bins{}nps.root".format(1, 1000, bins, 0))
    w = infile.Get("BinnedWorkspace")
    #for i in range(3):    
    start = time.time()
    NLL(w,1)
    end = time.time()
    av_time.append(float(end) - float(start))
    bintimes.append([sum(av_time)/float(len(av_time)),bins])
    infile.Close()

In [ ]:
chantimes = []
for chans in [1,2,3,4,5]:
    #for bins in [1,10,100]:
    #        for nps in [0,1,3]:
    av_time = []
    infile = ROOT.TFile.Open("output/workspace{}channels{}events{}bins{}nps.root".format(chans, 1000, 1, 1))
    w = infile.Get("BinnedWorkspace")
    #for i in range(5):
    start = time.time()
    NLL(w,1)
    end = time.time()
    av_time.append(float(end) - float(start))

    chantimes.append([sum(av_time)/float(len(av_time)),chans])
    infile.Close()

In [ ]:
nptimes = []
for nps in range(10):
    av_time = []
    for i in range(5):
        infile = ROOT.TFile.Open("output/workspace{}channels{}events{}bins{}nps.root".format(1, 1000, 1, nps))
        w = infile.Get("BinnedWorkspace")
        start = time.time()
        NLL(w,1)
        end = time.time()
        av_time.append(float(end) - float(start))
        infile.Close()
    nptimes.append([sum(av_time)/float(len(av_time)),nps])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
etimes = [(t[0]) for t in eventtimes]
btimes = [(t[0]) for t in bintimes]
ctimes = [(t[0]) for t in chantimes]
ntimes = [(t[0]) for t in nptimes]
plt.plot(etimes)
plt.plot(btimes)
plt.plot(ctimes)
plt.plot(ntimes)
plt.show()